# Pseudocode

 ## Cleaning:
    - NaN/?/Blank
        - waterfront
        - view
        - yr_rennovated
        - sqft_basement
    - whitespace *no issue
    - format to int/float
        - view
        - waterfront
        - condition
        - grade
        - sqft_basement
    - deal with dupes
        - several properties are listed more than once, suggesting they were bought/sold during this timeframe. 
    - categorical encoding
        - bedrooms
        - bathrooms
        - floors
        - zipcode
        - waterfront
        - view
        - condition
        - grade
    - duplicate properties
        - Located, need to decide how to treat. 
    - drop unused columns
        - lat/long once done with all other steps. Technically this combination is categorical.
        - date
        - sqft_basement? - 454 records do not have a value here--we would need to drop these records at a minimum.
    - save cleaned data
    
    
 ## Simple Model
     - add features
     - transform features (log transform?)
     
     
     
 ## 
     
     
     

## Ideas for stakeholders:
 - looking for properties to flip
    - Maybe look at adding sqft or bedrooms/bathrooms to see what improvements add most value?
 - looking for investment properties
     - identify combinations of bedrooms/bathrooms/year built/etc that are underpriced?
 - real estate agents providing guidance to sellers about what price they can expect for their home.
     - define ranges of price expected based on bedrooms/bathrooms/sqft/etc.
    

# Imports

In [1]:
# Importing packages

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import statsmodels.api as sm

from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# Simple Data - omitted columns recommended in project description


In [2]:
simple_housing = pd.read_csv('../data/kc_house_data.csv')

In [3]:
complexities = ['date', 'view', 'sqft_above', 'sqft_basement', 'yr_renovated', 'zipcode', 'lat', 'long', 'sqft_living15', 'sqft_lot15']

In [4]:
simple_housing.drop(columns=complexities, inplace=True)

In [5]:
simple_housing.head()

,id,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,condition,grade,yr_built
0,7129300520,221900.0,3,1.00,1180,5650,1.0,NaN,Average,7 Average,1955
1,6414100192,538000.0,3,2.25,2570,7242,2.0,NO,Average,7 Average,1951
2,5631500400,180000.0,2,1.00,770,10000,1.0,NO,Average,6 Low Average,1933
3,2487200875,604000.0,4,3.00,1960,5000,1.0,NO,Very Good,7 Average,1965
4,1954400510,510000.0,3,2.00,1680,8080,1.0,NO,Average,8 Good,1987


In [6]:
# Add price per square foot feature
simple_housing['price_sqft'] = simple_housing['price'] / simple_housing['sqft_living']

In [7]:
# drop waterfront & confirm no NaNs
simple_housing = simple_housing.drop(columns='waterfront')
simple_housing.isna().sum()

id             0
price          0
bedrooms       0
bathrooms      0
sqft_living    0
sqft_lot       0
floors         0
condition      0
grade          0
yr_built       0
price_sqft     0
dtype: int64

In [8]:
# think about binning if we're going categorically. Sounds like numerical

In [9]:
simple_housing.head()

,id,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,condition,grade,yr_built,price_sqft
0,7129300520,221900.0,3,1.00,1180,5650,1.0,Average,7 Average,1955,188.050847
1,6414100192,538000.0,3,2.25,2570,7242,2.0,Average,7 Average,1951,209.338521
2,5631500400,180000.0,2,1.00,770,10000,1.0,Average,6 Low Average,1933,233.766234
3,2487200875,604000.0,4,3.00,1960,5000,1.0,Very Good,7 Average,1965,308.163265
4,1954400510,510000.0,3,2.00,1680,8080,1.0,Average,8 Good,1987,303.571429


In [10]:
# Identify catgeorical values & create new df with only the categorical columns.
categoricals = ['condition','grade']
cat_df = simple_housing[categoricals]
cat_df.head()

,condition,grade
0,Average,7 Average
1,Average,7 Average
2,Average,6 Low Average
3,Very Good,7 Average
4,Average,8 Good


In [11]:
# OneHot encoding for categorical variables. Drop first column to avoid perfect multicolinearity.
ohe = OneHotEncoder(drop='first')
categorical_cats = ohe.fit_transform(cat_df)
categorical_cats.todense()
categorical_df = pd.DataFrame(categorical_cats.todense(), columns=ohe.get_feature_names(categoricals))
categorical_df.head()

,condition_Fair,condition_Good,condition_Poor,condition_Very Good,grade_11 Excellent,grade_12 Luxury,grade_13 Mansion,grade_3 Poor,grade_4 Low,grade_5 Fair,grade_6 Low Average,grade_7 Average,grade_8 Good,grade_9 Better
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [12]:
# Confirm identical # of records before concatenating.
print(categorical_df.shape)
print(simple_housing.shape)

(21597, 14)
(21597, 11)


In [13]:
# Drop origical categorical columns, concatenate encoded columns.
data = simple_housing.drop(columns=categoricals)
kc_encoded = pd.concat([data.reset_index(drop=True), categorical_df.reset_index(drop=True)], axis=1)

In [14]:
# Confirm no new records added & categoricals are gone.
print(kc_encoded.shape)
kc_encoded.head()

(21597, 23)


,id,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,yr_built,price_sqft,condition_Fair,...,grade_11 Excellent,grade_12 Luxury,grade_13 Mansion,grade_3 Poor,grade_4 Low,grade_5 Fair,grade_6 Low Average,grade_7 Average,grade_8 Good,grade_9 Better
0,7129300520,221900.0,3,1.00,1180,5650,1.0,1955,188.050847,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,6414100192,538000.0,3,2.25,2570,7242,2.0,1951,209.338521,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,5631500400,180000.0,2,1.00,770,10000,1.0,1933,233.766234,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,2487200875,604000.0,4,3.00,1960,5000,1.0,1965,308.163265,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,1954400510,510000.0,3,2.00,1680,8080,1.0,1987,303.571429,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [15]:
kc_encoded.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21597 entries, 0 to 21596
Data columns (total 23 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   21597 non-null  int64  
 1   price                21597 non-null  float64
 2   bedrooms             21597 non-null  int64  
 3   bathrooms            21597 non-null  float64
 4   sqft_living          21597 non-null  int64  
 5   sqft_lot             21597 non-null  int64  
 6   floors               21597 non-null  float64
 7   yr_built             21597 non-null  int64  
 8   price_sqft           21597 non-null  float64
 9   condition_Fair       21597 non-null  float64
 10  condition_Good       21597 non-null  float64
 11  condition_Poor       21597 non-null  float64
 12  condition_Very Good  21597 non-null  float64
 13  grade_11 Excellent   21597 non-null  float64
 14  grade_12 Luxury      21597 non-null  float64
 15  grade_13 Mansion     21597 non-null 

In [16]:
kc_encoded.to_csv('../data/kc_encoded.csv')

In [19]:
X = kc_encoded.drop(columns=['price_sqft'])
y = kc_encoded['price_sqft']
model = sm.OLS(endog=y, exog=X).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             price_sqft   R-squared:                       0.777
Model:                            OLS   Adj. R-squared:                  0.777
Method:                 Least Squares   F-statistic:                     3587.
Date:                Tue, 21 Jun 2022   Prob (F-statistic):               0.00
Time:                        11:05:37   Log-Likelihood:            -1.1594e+05
No. Observations:               21597   AIC:                         2.319e+05
Df Residuals:                   21575   BIC:                         2.321e+05
Df Model:                          21                                         
Covariance Type:            nonrobust                                         
=======================================================================================
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
id                   2.096e-10   1.24e-10      1.689      0.091   -3.37e-11    4.53e-10
price                   0.0004   1.51e-06    249.678      0.000       0.000       0.000
bedrooms               -4.6802      0.496     -9.430      0.000      -5.653      -3.707
bathrooms             -11.8395      0.796    -14.874      0.000     -13.400     -10.279
sqft_living            -0.1094      0.001   -128.605      0.000      -0.111      -0.108
sqft_lot             5.232e-05   8.84e-06      5.920      0.000     3.5e-05    6.96e-05
floors                  3.5189      0.823      4.274      0.000       1.905       5.133
yr_built                0.1663      0.002    101.827      0.000       0.163       0.169
condition_Fair        -15.9924      4.050     -3.949      0.000     -23.931      -8.054
condition_Good          2.6640      0.863      3.087      0.002       0.972       4.356
condition_Poor          1.5800      9.726      0.162      0.871     -17.484      20.644
condition_Very Good     8.4609      1.381      6.126      0.000       5.754      11.168
grade_11 Excellent    -20.6828      3.109     -6.652      0.000     -26.777     -14.588
grade_12 Luxury      -100.2016      5.994    -16.717      0.000    -111.950     -88.453
grade_13 Mansion     -348.3636     15.025    -23.186      0.000    -377.813    -318.914
grade_3 Poor          143.3306     52.008      2.756      0.006      41.392     245.269
grade_4 Low            28.9247     10.305      2.807      0.005       8.726      49.123
grade_5 Fair          -11.5155      4.090     -2.815      0.005     -19.533      -3.498
grade_6 Low Average   -14.8847      2.521     -5.905      0.000     -19.825      -9.944
grade_7 Average       -11.9349      2.112     -5.650      0.000     -16.075      -7.795
grade_8 Good           -0.9745      1.940     -0.502      0.615      -4.777       2.828
grade_9 Better          7.7521      1.913      4.052      0.000       4.002      11.502
==============================================================================
Omnibus:                     5756.271   Durbin-Watson:                   1.985
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           517139.053
Skew:                           0.177   Prob(JB):                         0.00
Kurtosis:                      26.970   Cond. No.                     7.96e+11
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 7.96e+11. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

# Full EDA

In [ ]:
kc_housing = pd.read_csv('../data/kc_house_data.csv')


In [ ]:
kc_housing.shape

In [ ]:
kc_housing.describe()

In [ ]:
columns_of_note = list(kc_housing.columns)[3:8]

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=3, figsize=(16,16))


for xcol, ax in zip(kc_housing.columns[3:8], [x for v in axes for x in v]):
    kc_housing.plot.scatter(x=xcol, y='price', ax=ax, alpha=0.5, color='b')


In [ ]:
import warnings
warnings.filterwarnings('ignore')
fig = plt.figure(figsize = (15,15))
ax = fig.gca()
kc_housing.hist(ax = ax);

In [ ]:
kc_housing.info()

In [ ]:
# Looking for NANs
kc_housing.isna().sum()

#### Added price/sqft feature

In [ ]:
kc_housing['price_sqft'] = kc_housing['price']/kc_housing['sqft_living']

In [ ]:
kc_housing.head()

#### Addressing Waterfront

There are ~2400 NA cells, no easy way to determine if waterfront or not. Recommend we drop this whole column--even if all of the NA cells should be YES for waterfront, that's less than 10% of our dataset, likely not meaningful to our analysis.

Drop Waterfront.

In [ ]:
kc_housing['waterfront'].value_counts()

In [ ]:
sum(kc_housing['waterfront'].isna())

In [ ]:
kc_housing['waterfront'] = kc_housing['waterfront'].fillna('N/A')

In [ ]:
kc_housing.sort_values(by='waterfront', ascending=False)

#### Addressing View

Doesn't seem to be many rows with na. Maybe encode as NA or preferably drop?+

Drop rows with N/A

In [ ]:
sum(kc_housing['view'].isna())

In [ ]:
kc_housing['view'].value_counts()

In [ ]:
# code to replace empty with N/A - add inplace if we decide to take this approach.
kc_housing['view'] = kc_housing['view'].fillna('N/A')

#### Addressing yr_renovated

Absent from simple data, maybe we add on later

~ 78% have 0 values, so assume they were not rennovated. I think rennovations are a meaningful datapoint, so we should encode this somehow. We could think about how recent the rennovation was to see if there is a meaningful relationship--for example, a home rennovated in 2020 is likely to sell better than a home rennovated only in 1980. Maybe we do some research on this and decide on a breakpoint for rennovated before/after? We could do a simple linear regression between yr renovated and price...

In [ ]:
sum(kc_housing['yr_renovated'].isna())

In [ ]:
kc_housing['yr_renovated'].value_counts(sort=False).head(75)

In [ ]:
kc_housing['yr_renovated'] = kc_housing['yr_renovated'].fillna('N/A')

In [ ]:
#dropping N/A for now to make model function.
kc_housing = kc_housing[(kc_housing.yr_renovated != 'N/A')]

#### Duplicate Properties

In [ ]:
# Looking for duplicate properties

kc_housing['id'].duplicated().sum()

In [ ]:
# Identify instances of the same property appearing more than once in the data set based on lattitude, longitude
# How do we treat this? Might not matter since there is only ~4-5 months of data here. as long as listing date 
# is different, probably OK to treat these as unique listings.


kc_housing[kc_housing.duplicated(subset=['id'], keep=False)].sort_values('id')

#### Addressing grade

This we need to encode as an ordinal.

In [ ]:
kc_housing['grade'].value_counts()

In [ ]:
# Create new column for numerical grade. 
# Should either drop df['grade'] or omit it from clean df.
kc_housing['grade_num']= kc_housing['grade'].apply(lambda x: x[:1]).astype(int)

In [ ]:
kc_housing['grade_num']

In [ ]:
kc_housing.info()

#### Addressing sqft_basement

Recommend we drop rows with N/A since we don't have an easy way to encode.
We have 454 unknown values here. need to drop or encode somehow. Maybe has basement or no basement?


In [ ]:
kc_housing['sqft_basement'].value_counts()

In [ ]:
kc_housing = kc_housing[(kc_housing.sqft_basement != '?')]

In [ ]:
kc_housing['sqft_basement'].value_counts()

# Encoding Categoricals

In [ ]:
# Identify catgeorical values & create new df with only the categorical columns.
categoricals = ['bedrooms', 'floors', 'condition', 'grade', 'view','waterfront']
cat_df = kc_housing[categoricals]
cat_df.head()

In [ ]:
# OneHot encoding for categorical variables. Drop first column to avoid perfect multicolinearity.
ohe = OneHotEncoder(drop='first')
categorical_cats = ohe.fit_transform(cat_df)
categorical_cats.todense()
categorical_df = pd.DataFrame(categorical_cats.todense(), columns=ohe.get_feature_names(categoricals))
categorical_df.head()

In [ ]:
categorical_df.shape

In [ ]:
kc_housing.shape

In [ ]:
# Append categorical dummies to original dataframe, and drop origical categorical columns.
data = kc_housing.drop(columns=categoricals)
kc_encoded = pd.concat([data.reset_index(drop=True), categorical_df.reset_index(drop=True)], axis=1)

In [ ]:
kc_encoded.shape

In [ ]:
kc_encoded.info()

# Nonsense Model with all columns

In [ ]:
X = kc_encoded.drop(columns=['date', 'lat', 'long', 'id','price_sqft','yr_renovated','sqft_basement'])
y = kc_encoded['price_sqft']

In [ ]:
X.isna().sum()

In [ ]:

model = sm.OLS(endog=y, exog=X).fit()
model.summary()